# Config

In [ ]:
#!pip install boto3
#from IPython.display import clear_output
#clear_output(wait=True)

In [ ]:
import json

def init(config_file):
    response = {}
    with open(config_file) as json_file: 
        response = json.load(json_file)        
    return response

# Class for AWS S3

In [ ]:
import boto3
import os

class AWSS3:
    def __init__(self, config):
        self._config = config
        self._s3_client = self.s3_client()
        self._s3_resource = self.s3_resource()
        

    def s3_client(self):
        return boto3.client(
            's3',
            aws_access_key_id = self._config['ACCESS_KEY'],
            aws_secret_access_key = self._config['SECRET_KEY'],
            region_name = self._config['REGION']
        )
    
    
    def s3_resource(self):
        return boto3.resource(
            's3',
            aws_access_key_id = self._config['ACCESS_KEY'],
            aws_secret_access_key = self._config['SECRET_KEY'],
            region_name = self._config['REGION']
        )
    
    
    def list_buckets(self):
        return self._s3_client.list_buckets()
    
    
    def list_s3_objects_bucket(self, bucket_name: str):
        response = {}
        try:
            response = self._s3_client.list_objects(Bucket = bucket_name)
        except ClientError as e:
            logging.error(e)
            return response
        
        for bucket in response['Contents']:
                print(bucket["Key"])
                
        return response
    
    
    def get_bucket_policy(self, bucket_name):
        return self._s3_client.get_bucket_policy(Bucket=bucket_name)
    

    def create_bucket(self, bucket_name, location = ''):
        try: 
            response = {}
            if((len(location) == 0) or (location == 'us-east-1')):
                response = self._s3_client.create_bucket(Bucket=bucket_name)
            else:
                response = self._s3_client.create_bucket(
                    Bucket=bucket_name,
                    CreateBucketConfiguration={
                        'LocationConstraint': location
                    }
                )
        except Exception as e:
            print("Oops!", e.__class__, "occurred.")        
        return response
    
    
    def delete_bucket(self, bucket_name):
        return self._s3_client.delete_bucket(Bucket=bucket_name)
    
    
    def delete_object(self, bucket_name, key_name):
     res = self._s3_client.delete_object(Bucket=bucket_name, Key=key_name)
    
    
    def update_bucket_policy(self, bucket_name):
        bucket_policy = {
            'Version': '2012-10-17',
            'Statement': [
                {
                    'Sid': 'AddPerm',
                    'Effect': 'Allow',
                    'Principal': '*',
                    'Action': [
                        's3:DeleteObject',
                        's3:GetObject',
                        's3:PutObject'
                    ],
                    'Resource': 'arn:aws:s3:::' + bucket_name + '/*'
                }
            ]
        }

        policy_string = json.dumps(bucket_policy)

        return self._s3_client.put_bucket_policy(
            Bucket=bucket_name,
            Policy=policy_string
        )
    
    
    def upload_file(self, bucket_name, file_path, key_name, content_type):
        return self._s3_client.upload_file(file_path, bucket_name, key_name,
                                           ExtraArgs={'ACL': 'public-read',
                                                      'ContentType': content_type,
                                                      'ContentDisposition': 'inline'
                                                     }
                                          )
    


In [ ]:
config = init('./aws.config')
aws_s3 = AWSS3(config)

In [ ]:
#aws_s3.create_bucket('test19802021', 'us-west-2')
#aws_s3.create_bucket('test19802022', 'us-west-2')
aws_s3.create_bucket('test19802020', 'us-east-1')
aws_s3.create_bucket('test19802021')

In [ ]:
aws_s3.update_bucket_policy('test19802020')
aws_s3.upload_file('test19802020', os.getcwd() + os.path.sep + 'documents' + os.path.sep + 'index.html',
                   'index.html', 'text/html')
aws_s3.upload_file('test19802020', os.getcwd() + os.path.sep + 'documents' + os.path.sep + 'page2.html',
                   'page2.html', 'text/html')
aws_s3.upload_file('test19802020', os.getcwd() + os.path.sep + 'documents' + os.path.sep + '0.jpeg',
                   '0.jpeg', 'image/jpeg')


In [ ]:
aws_s3.delete_object('test19802020', 'page2.html')
aws_s3.delete_bucket('test19802021')